# 準備

## ディレクトリを移動

以下では，Googleドライブのマイドライブ直下にDNN_codeフォルダを置くことを仮定しています．必要に応じて，パスを変更してください．

In [1]:
import os
os.chdir('drive/My Drive/DNN_code/lesson_3/3_2_tf_languagemodel/')

FileNotFoundError: [Errno 2] No such file or directory: 'drive/My Drive/DNN_code/lesson_3/3_2_tf_languagemodel/'

# predict_word

In [8]:
import tensorflow as tf
import numpy as np
import re
import glob
import collections
import random
import pickle
import time
import datetime
import os

# logging levelを変更
tf.logging.set_verbosity(tf.logging.ERROR)

class Corpus:
    def __init__(self):
        self.unknown_word_symbol = "<???>" # 出現回数の少ない単語は未知語として定義しておく
        self.unknown_word_threshold = 3 # 未知語と定義する単語の出現回数の閾値
        self.corpus_file = "./corpus/**/*.txt"
        self.corpus_encoding = "utf-8"
        self.dictionary_filename = "./data_for_predict/word_dict.dic"
        self.chunk_size = 5
        self.load_dict()

        words = []
        for filename in glob.glob(self.corpus_file, recursive=True):
            with open(filename, "r", encoding=self.corpus_encoding) as f:

                # word breaking
                text = f.read()
                # 全ての文字を小文字に統一し、改行をスペースに変換
                text = text.lower().replace("\n", " ")
                # 特定の文字以外の文字を空文字に置換する
                text = re.sub(r"[^a-z '\-]", "", text)
                # 複数のスペースはスペース一文字に変換
                text = re.sub(r"[ ]+", " ", text)

                # 前処理： '-' で始まる単語は無視する
                words = [ word for word in text.split() if not word.startswith("-")]


        self.data_n = len(words) - self.chunk_size
        self.data = self.seq_to_matrix(words)

    def prepare_data(self):
        """
        訓練データとテストデータを準備する。
        data_n = ( text データの総単語数 ) - chunk_size
        input: (data_n, chunk_size, vocabulary_size)
        output: (data_n, vocabulary_size)
        """

        # 入力と出力の次元テンソルを準備
        all_input = np.zeros([self.chunk_size, self.vocabulary_size, self.data_n])
        all_output = np.zeros([self.vocabulary_size, self.data_n])

        # 準備したテンソルに、コーパスの one-hot 表現(self.data) のデータを埋めていく
        # i 番目から ( i + chunk_size - 1 ) 番目までの単語が１組の入力となる
        # このときの出力は ( i + chunk_size ) 番目の単語
        for i in range(self.data_n):
            all_output[:, i] = self.data[:, i + self.chunk_size] # (i + chunk_size) 番目の単語の one-hot ベクトル
            for j in range(self.chunk_size):
                all_input[j, :, i] = self.data[:, i + self.chunk_size - j - 1]

        # 後に使うデータ形式に合わせるために転置を取る
        all_input = all_input.transpose([2, 0, 1])
        all_output = all_output.transpose()

        # 訓練データ：テストデータを 4 : 1 に分割する
        training_num = ( self.data_n * 4 ) // 5
        return all_input[:training_num], all_output[:training_num], all_input[training_num:], all_output[training_num:]


    def build_dict(self):
        # コーパス全体を見て、単語の出現回数をカウントする
        counter = collections.Counter()
        for filename in glob.glob(self.corpus_file, recursive=True):
            with open(filename, "r", encoding=self.corpus_encoding) as f:

                # word breaking
                text = f.read()
                # 全ての文字を小文字に統一し、改行をスペースに変換
                text = text.lower().replace("\n", " ")
                # 特定の文字以外の文字を空文字に置換する
                text = re.sub(r"[^a-z '\-]", "", text)
                # 複数のスペースはスペース一文字に変換
                text = re.sub(r"[ ]+", " ", text)

                # 前処理： '-' で始まる単語は無視する
                words = [word for word in text.split() if not word.startswith("-")]

                counter.update(words)

        # 出現頻度の低い単語を一つの記号にまとめる
        word_id = 0
        dictionary = {}
        for word, count in counter.items():
            if count <= self.unknown_word_threshold:
                continue

            dictionary[word] = word_id
            word_id += 1
        dictionary[self.unknown_word_symbol] = word_id

        print("総単語数：", len(dictionary))

        # 辞書を pickle を使って保存しておく
        with open(self.dictionary_filename, "wb") as f:
            pickle.dump(dictionary, f)
            print("Dictionary is saved to", self.dictionary_filename)

        self.dictionary = dictionary

        print(self.dictionary)

    def load_dict(self):
        with open(self.dictionary_filename, "rb") as f:
            self.dictionary = pickle.load(f)
            self.vocabulary_size = len(self.dictionary)
            self.input_layer_size = len(self.dictionary)
            self.output_layer_size = len(self.dictionary)
            print("総単語数: ", self.input_layer_size)

    def get_word_id(self, word):
        # print(word)
        # print(self.dictionary)
        # print(self.unknown_word_symbol)
        # print(self.dictionary[self.unknown_word_symbol])
        # print(self.dictionary.get(word, self.dictionary[self.unknown_word_symbol]))
        return self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])

    # 入力された単語を one-hot ベクトルにする
    def to_one_hot(self, word):
        index = self.get_word_id(word)
        data = np.zeros(self.vocabulary_size)
        data[index] = 1
        return data

    def seq_to_matrix(self, seq):
        print(seq)
        data = np.array([self.to_one_hot(word) for word in seq]) # (data_n, vocabulary_size)
        return data.transpose() # (vocabulary_size, data_n)

class Language:
    """
    input layer: self.vocabulary_size
    hidden layer: rnn_size = 30
    output layer: self.vocabulary_size
    """

    def __init__(self):
        self.corpus = Corpus()
        self.dictionary = self.corpus.dictionary
        self.vocabulary_size = len(self.dictionary) # 単語数
        self.input_layer_size = self.vocabulary_size # 入力層の数
        self.hidden_layer_size = 30 # 隠れ層の RNN ユニットの数
        self.output_layer_size = self.vocabulary_size # 出力層の数
        self.batch_size = 128 # バッチサイズ
        self.chunk_size = 5 # 展開するシーケンスの数。c_0, c_1, ..., c_(chunk_size - 1) を入力し、c_(chunk_size) 番目の単語の確率が出力される。
        self.learning_rate = 0.005 # 学習率
        self.epochs = 1000 # 学習するエポック数
        self.forget_bias = 1.0 # LSTM における忘却ゲートのバイアス
        self.model_filename = "./data_for_predict/predict_model.ckpt"
        self.unknown_word_symbol = self.corpus.unknown_word_symbol

    def inference(self, input_data, initial_state):
        """
        :param input_data: (batch_size, chunk_size, vocabulary_size) 次元のテンソル
        :param initial_state: (batch_size, hidden_layer_size) 次元の行列
        :return:
        """
        # 重みとバイアスの初期化
        hidden_w = tf.Variable(tf.truncated_normal([self.input_layer_size, self.hidden_layer_size], stddev=0.01))
        hidden_b = tf.Variable(tf.ones([self.hidden_layer_size]))
        output_w = tf.Variable(tf.truncated_normal([self.hidden_layer_size, self.output_layer_size], stddev=0.01))
        output_b = tf.Variable(tf.ones([self.output_layer_size]))

        # BasicLSTMCell, BasicRNNCell は (batch_size, hidden_layer_size) が chunk_size 数ぶんつながったリストを入力とする。
        # 現時点での入力データは (batch_size, chunk_size, input_layer_size) という３次元のテンソルなので
        # tf.transpose や tf.reshape などを駆使してテンソルのサイズを調整する。

        input_data = tf.transpose(input_data, [1, 0, 2]) # 転置。(chunk_size, batch_size, vocabulary_size)
        input_data = tf.reshape(input_data, [-1, self.input_layer_size]) # 変形。(chunk_size * batch_size, input_layer_size)
        input_data = tf.matmul(input_data, hidden_w) + hidden_b # 重みWとバイアスBを適用。 (chunk_size, batch_size, hidden_layer_size)
        input_data = tf.split(input_data, self.chunk_size, 0) # リストに分割。chunk_size * (batch_size, hidden_layer_size)

        # RNN のセルを定義する。RNN Cell の他に LSTM のセルや GRU のセルなどが利用できる。
        with tf.variable_scope('auto_reuse', reuse=tf.AUTO_REUSE):
            #cell = tf.nn.rnn_cell.BasicRNNCell(self.hidden_layer_size)
            #cell = tf.nn.rnn_cell.BasicLSTMCell(self.hidden_layer_size)
            cell = tf.nn.rnn_cell.GRUCell(self.hidden_layer_size)
            outputs, states = tf.nn.static_rnn(cell, input_data, initial_state=initial_state)
        
        # 最後に隠れ層から出力層につながる重みとバイアスを処理する
        # 最終的に softmax 関数で処理し、確率として解釈される。
        # softmax 関数はこの関数の外で定義する。
        output = tf.matmul(outputs[-1], output_w) + output_b

        return output

    def loss(self, logits, labels):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

        return cost

    def training(self, cost):
        # 今回は最適化手法として Adam を選択する。
        # ここの AdamOptimizer の部分を変えることで、Adagrad, Adadelta などの他の最適化手法を選択することができる
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(cost)

        return optimizer

    def train(self):
        # 変数などの用意
        input_data = tf.placeholder("float", [None, self.chunk_size, self.input_layer_size])
        actual_labels = tf.placeholder("float", [None, self.output_layer_size])
        initial_state = tf.placeholder("float", [None, self.hidden_layer_size])

        prediction = self.inference(input_data, initial_state)
        cost = self.loss(prediction, actual_labels)
        optimizer = self.training(cost)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(actual_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

        # TensorBoard で可視化するため、クロスエントロピーをサマリーに追加
        tf.summary.scalar("Cross entropy: ", cost)
        summary = tf.summary.merge_all()

        # 訓練・テストデータの用意
        # corpus = Corpus()
        trX, trY, teX, teY = self.corpus.prepare_data()
        training_num = trX.shape[0]

        # ログを保存するためのディレクトリ
        timestamp = time.time()
        dirname = datetime.datetime.fromtimestamp(timestamp).strftime("%Y%m%d%H%M%S")

        # ここから実際に学習を走らせる
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            summary_writer = tf.summary.FileWriter("./log/" + dirname, sess.graph)

            # エポックを回す
            for epoch in range(self.epochs):
                step = 0
                epoch_loss = 0
                epoch_acc = 0

                # 訓練データをバッチサイズごとに分けて学習させる (= optimizer を走らせる)
                # エポックごとの損失関数の合計値や（訓練データに対する）精度も計算しておく
                while (step + 1) * self.batch_size < training_num:
                    start_idx = step * self.batch_size
                    end_idx = (step + 1) * self.batch_size

                    batch_xs = trX[start_idx:end_idx, :, :]
                    batch_ys = trY[start_idx:end_idx, :]

                    _, c, a = sess.run([optimizer, cost, accuracy],
                                       feed_dict={input_data: batch_xs,
                                                  actual_labels: batch_ys,
                                                  initial_state: np.zeros([self.batch_size, self.hidden_layer_size])
                                                  }
                                       )
                    epoch_loss += c
                    epoch_acc += a
                    step += 1

                # コンソールに損失関数の値や精度を出力しておく
                print("Epoch", epoch, "completed ouf of", self.epochs, "-- loss:", epoch_loss, " -- accuracy:",
                      epoch_acc / step)

                # Epochが終わるごとにTensorBoard用に値を保存
                summary_str = sess.run(summary, feed_dict={input_data: trX,
                                                           actual_labels: trY,
                                                           initial_state: np.zeros(
                                                               [trX.shape[0],
                                                                self.hidden_layer_size]
                                                           )
                                                           }
                                       )
                summary_writer.add_summary(summary_str, epoch)
                summary_writer.flush()

            # 学習したモデルも保存しておく
            saver = tf.train.Saver()
            saver.save(sess, self.model_filename)

            # 最後にテストデータでの精度を計算して表示する
            a = sess.run(accuracy, feed_dict={input_data: teX, actual_labels: teY,
                                              initial_state: np.zeros([teX.shape[0], self.hidden_layer_size])})
            print("Accuracy on test:", a)


    def predict(self, seq):
        """
        文章を入力したときに次に来る単語を予測する
        :param seq: 予測したい単語の直前の文字列。chunk_size 以上の単語数が必要。
        :return:
        """

        # 最初に復元したい変数をすべて定義してしまいます
        tf.reset_default_graph()
        input_data = tf.placeholder("float", [None, self.chunk_size, self.input_layer_size])
        initial_state = tf.placeholder("float", [None, self.hidden_layer_size])
        prediction = tf.nn.softmax(self.inference(input_data, initial_state))
        predicted_labels = tf.argmax(prediction, 1)

        # 入力データの作成
        # seq を one-hot 表現に変換する。
        words = [word for word in seq.split() if not word.startswith("-")]
        x = np.zeros([1, self.chunk_size, self.input_layer_size])
        for i in range(self.chunk_size):
            word = seq[len(words) - self.chunk_size + i]
            index = self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])
            x[0][i][index] = 1
        feed_dict = {
            input_data: x, # (1, chunk_size, vocabulary_size)
            initial_state: np.zeros([1, self.hidden_layer_size])
        }

        # tf.Session()を用意
        with tf.Session() as sess:
            # 保存したモデルをロードする。ロード前にすべての変数を用意しておく必要がある。
            saver = tf.train.Saver()
            saver.restore(sess, self.model_filename)

            # ロードしたモデルを使って予測結果を計算
            u, v = sess.run([prediction, predicted_labels], feed_dict=feed_dict)

            keys = list(self.dictionary.keys())


            # コンソールに文字ごとの確率を表示
            for i in range(self.vocabulary_size):
                c = self.unknown_word_symbol if i == (self.vocabulary_size - 1) else keys[i]
                print(c, ":", u[0][i])

            print("Prediction:", seq + " " + ("<???>" if v[0] == (self.vocabulary_size - 1) else keys[v[0]]))

        return u[0]


def build_dict():
    cp = Corpus()
    cp.build_dict()

if __name__ == "__main__":
    build_dict()

    ln = Language()

    # 学習するときに呼び出す
    ln.train()

    # 保存したモデルを使って単語の予測をする
    ln.predict("some of them looks like")


総単語数:  9826
['welcome', 'back', 'to', 'our', 'show', 'ok', 'this', 'is', 'maria', 'and', 'i', "don't", 'know', 'your', 'last', 'name', 'polk', 'oh', "that's", 'good', 'from', 'polkton', 'maria', 'polk', 'ok', 'and', 'where', 'did', 'you', 'grow', 'up', 'were', 'you', 'born', 'in', 'this', 'area', 'here', 'i', 'was', 'born', 'in', 'cottonville', 'right', 'outside', 'of', 'norwood', 'in', 'stanley', 'county', 'oh', 'ok', 'when', 'you', 'were', 'little', 'did', 'your', 'mom', 'read', 'you', 'books', 'or', 'did', 'somebody', 'read', 'you', 'books', 'in', 'your', 'house', 'my', 'mama', 'read', 'to', 'me', 'and', 'my', 'sister', 'uh-huh', 'what', 'kind', 'of', 'books', 'were', 'your', 'favorites', 'do', 'you', 'remember', 'any', 'that', 'were', 'maybe', 'a', 'real', 'favorite', 'of', 'yours', 'mm', 'i', 'kind', 'of', 'liked', 'them', 'all', 'um', 'i', "didn't", 'really', 'have', 'a', 'favorite', 'no', 'any', 'particular', 'types', 'like', 'ones', 'with', 'pictures', 'or', 'can', 'you', 'thin

['welcome', 'back', 'to', 'our', 'show', 'ok', 'this', 'is', 'maria', 'and', 'i', "don't", 'know', 'your', 'last', 'name', 'polk', 'oh', "that's", 'good', 'from', 'polkton', 'maria', 'polk', 'ok', 'and', 'where', 'did', 'you', 'grow', 'up', 'were', 'you', 'born', 'in', 'this', 'area', 'here', 'i', 'was', 'born', 'in', 'cottonville', 'right', 'outside', 'of', 'norwood', 'in', 'stanley', 'county', 'oh', 'ok', 'when', 'you', 'were', 'little', 'did', 'your', 'mom', 'read', 'you', 'books', 'or', 'did', 'somebody', 'read', 'you', 'books', 'in', 'your', 'house', 'my', 'mama', 'read', 'to', 'me', 'and', 'my', 'sister', 'uh-huh', 'what', 'kind', 'of', 'books', 'were', 'your', 'favorites', 'do', 'you', 'remember', 'any', 'that', 'were', 'maybe', 'a', 'real', 'favorite', 'of', 'yours', 'mm', 'i', 'kind', 'of', 'liked', 'them', 'all', 'um', 'i', "didn't", 'really', 'have', 'a', 'favorite', 'no', 'any', 'particular', 'types', 'like', 'ones', 'with', 'pictures', 'or', 'can', 'you', 'think', 'of', 'a

Epoch 19 completed ouf of 1000 -- loss: 14.020723342895508  -- accuracy: 0.052083333333333336
Epoch 20 completed ouf of 1000 -- loss: 14.01825761795044  -- accuracy: 0.052083333333333336
Epoch 21 completed ouf of 1000 -- loss: 14.016261100769043  -- accuracy: 0.052083333333333336
Epoch 22 completed ouf of 1000 -- loss: 14.014068126678467  -- accuracy: 0.052083333333333336
Epoch 23 completed ouf of 1000 -- loss: 14.011593341827393  -- accuracy: 0.052083333333333336
Epoch 24 completed ouf of 1000 -- loss: 14.009069442749023  -- accuracy: 0.052083333333333336
Epoch 25 completed ouf of 1000 -- loss: 14.006770133972168  -- accuracy: 0.052083333333333336
Epoch 26 completed ouf of 1000 -- loss: 14.004835605621338  -- accuracy: 0.052083333333333336
Epoch 27 completed ouf of 1000 -- loss: 14.003242015838623  -- accuracy: 0.052083333333333336
Epoch 28 completed ouf of 1000 -- loss: 14.001878261566162  -- accuracy: 0.052083333333333336
Epoch 29 completed ouf of 1000 -- loss: 14.000657081604004  -

Epoch 107 completed ouf of 1000 -- loss: 13.974126815795898  -- accuracy: 0.052083333333333336
Epoch 108 completed ouf of 1000 -- loss: 13.974021434783936  -- accuracy: 0.052083333333333336
Epoch 109 completed ouf of 1000 -- loss: 13.973917961120605  -- accuracy: 0.052083333333333336
Epoch 110 completed ouf of 1000 -- loss: 13.973817348480225  -- accuracy: 0.052083333333333336
Epoch 111 completed ouf of 1000 -- loss: 13.973717212677002  -- accuracy: 0.052083333333333336
Epoch 112 completed ouf of 1000 -- loss: 13.973619937896729  -- accuracy: 0.052083333333333336
Epoch 113 completed ouf of 1000 -- loss: 13.973524570465088  -- accuracy: 0.052083333333333336
Epoch 114 completed ouf of 1000 -- loss: 13.97343111038208  -- accuracy: 0.052083333333333336
Epoch 115 completed ouf of 1000 -- loss: 13.97333812713623  -- accuracy: 0.052083333333333336
Epoch 116 completed ouf of 1000 -- loss: 13.973247528076172  -- accuracy: 0.052083333333333336
Epoch 117 completed ouf of 1000 -- loss: 13.97315835

Epoch 194 completed ouf of 1000 -- loss: 13.969192028045654  -- accuracy: 0.052083333333333336
Epoch 195 completed ouf of 1000 -- loss: 13.969162940979004  -- accuracy: 0.052083333333333336
Epoch 196 completed ouf of 1000 -- loss: 13.969133853912354  -- accuracy: 0.052083333333333336
Epoch 197 completed ouf of 1000 -- loss: 13.96910572052002  -- accuracy: 0.052083333333333336
Epoch 198 completed ouf of 1000 -- loss: 13.969077110290527  -- accuracy: 0.052083333333333336
Epoch 199 completed ouf of 1000 -- loss: 13.969049453735352  -- accuracy: 0.052083333333333336
Epoch 200 completed ouf of 1000 -- loss: 13.969021320343018  -- accuracy: 0.052083333333333336
Epoch 201 completed ouf of 1000 -- loss: 13.968994140625  -- accuracy: 0.052083333333333336
Epoch 202 completed ouf of 1000 -- loss: 13.968966960906982  -- accuracy: 0.052083333333333336
Epoch 203 completed ouf of 1000 -- loss: 13.968940734863281  -- accuracy: 0.052083333333333336
Epoch 204 completed ouf of 1000 -- loss: 13.9689130783

...

singled : 9.9021105e-05
soviet : 0.000103991915
accompanying : 0.00010165863
affidavit : 9.819394e-05
descriptions : 9.559488e-05
identities : 9.474284e-05
renewing : 9.8124925e-05
excerpts : 9.588877e-05
mobster : 0.00010366458
referring : 0.00010929052
familiarity : 9.6872885e-05
stunned : 0.00010892711
competitors : 0.00010179965
compete : 9.932427e-05
upjohn : 0.0001011758
cosmetic : 9.814227e-05
jonathan : 0.000104212835
shareholder : 0.00010524113
ala : 0.000103282626
vanderbilt : 0.0001041686
johnson : 0.000100139914
signing : 0.000100158424
tech : 9.89694e-05
t-shirts : 0.00010196414
scores : 0.000106103034
coach : 0.00010006642
cutcliffe : 9.82416e-05
weighs : 0.000101458056
rex : 0.000106183325
trophy : 0.00010307269
promotional : 0.00010109058
miami : 0.0001042743
dow : 0.00010349343
telerate : 0.000104660496
directors : 9.9190365e-05
magazine : 0.00010133017
admonished : 0.000100347315
torricelli : 0.000101372214
violated : 9.625288e-05
torricelli's : 9.356923e-05
chang : 0

ghost : 0.000103241975
roots : 0.00010554295
tentacles : 9.6202275e-05
crushing : 9.5149226e-05
kraken : 0.000100242316
god's : 0.000108848246
tia's : 0.000105132814
shack : 9.827562e-05
tia : 0.00010742006
dalma : 0.00011049694
claws : 9.632126e-05
hoped : 9.908538e-05
jar : 0.00010016002
sums : 9.9577184e-05
versions : 0.00010400226
wid : 9.566335e-05
beads : 0.00010917748
scuttled : 0.0001044983
reef : 0.00010001521
awaits : 9.7799544e-05
passes : 9.9314086e-05
crewmembers : 0.00010574534
swordfight : 9.8135104e-05
slices : 9.900014e-05
maccus : 0.00010754577
bare : 0.00010266214
ninety-nine : 0.00010326371
jones' : 9.7204174e-05
cabin : 0.00010869228
platform : 9.866203e-05
romantic : 0.0001063519
commodore : 0.00010332856
hired : 0.00010573408
habits : 0.00010589034
fires : 0.00010573677
heave : 0.000107294014
orderly : 9.527618e-05
bo'sun : 0.0001004749
cannon : 9.893264e-05
tentacle : 0.00010382292
compassion : 9.870776e-05
wyvern : 9.9934754e-05
stab : 0.000102084894
unhappy : 

gov't : 0.00010423221
teaparty : 9.6574775e-05
frequency : 0.000102192535
nintendo : 0.00010790425
wii : 9.7434975e-05
tweets : 9.5852534e-05
album : 0.00011236853
presidency : 0.00010379925
teabagging : 0.000104301806
webinar : 9.814321e-05
homosexuals : 9.990188e-05
haha : 0.00010224162
prevailing : 0.00010026223
thanx : 0.00010278703
marketers : 9.9112636e-05
ourselves : 9.776389e-05
graham : 0.000105589876
bandwidth : 9.9317615e-05
recipe : 9.753266e-05
libraries : 0.00011007188
fe : 0.00010249229
diapers : 0.00010955276
joins : 9.853317e-05
conservatives : 0.00010119522
denver : 0.00010079296
treats : 0.00010487588
nursing : 0.00010634504
nra : 0.00010159752
fox : 0.00010307796
tuition : 9.668437e-05
mason : 0.0001107904
trout : 9.55194e-05
directories : 0.000104901854
gossip : 9.823971e-05
guerrilla : 0.00010476559
appstore : 0.000100519894
ought : 0.00011436423
careers : 0.00010593907
kit : 9.978837e-05
shorten : 9.8024095e-05
scholarly : 0.000102742364
pp : 0.00010730864
anytim

warping : 0.000101271566
steepened : 0.00010615483
horst : 9.85392e-05
graben : 0.00010127132
basins : 9.9688405e-05
floodplain : 0.00010786244
steepening : 0.00010093214
aggradation : 0.00010639697
segment : 0.00010287481
steeper : 0.00010701494
blocked : 0.000104267354
sediment : 9.6904216e-05
tilt : 9.543281e-05
migrating : 9.4928495e-05
sinuous : 0.00010193885
meandering : 0.00010419348
braided : 0.000103161634
schumm : 0.00010181407
suspended-load : 9.828918e-05
mixed-load : 9.954882e-05
bed-load : 9.796962e-05
meander : 0.00010016448
sinuosity : 0.000101515616
thresholds : 0.000102831684
flume : 0.000105658626
tributary : 0.000105070125
tectonics : 0.00010687294
anastomosing : 0.00010711008
uplift : 0.00010126188
deformed : 9.7994765e-05
terraces : 9.8756616e-05
variations : 0.000100031335
dendritic : 0.00010416647
isostatic : 9.847217e-05
incision : 0.00010588248
chattahoochee : 9.826439e-05
subsidence : 0.00010168873
asymmetry : 9.5840376e-05
geological : 0.000107130014
contour

malaquez's : 0.00010624571
twenty-four : 9.661959e-05
lightscreen : 9.90632e-05
sarah's : 0.000102792736
couches : 0.00010629218
zheng : 0.00010296483
kishori : 9.403071e-05
day-to-day : 9.77674e-05
robots : 9.8397926e-05
adrienne : 9.757932e-05
hypothesis : 0.000108773165
youngstown : 0.000105126484
mahoning : 9.920485e-05
struthers : 0.000100941375
boardman : 0.000104220715
mills : 9.3218245e-05
youngstowns : 0.000111368354
uswa : 9.89443e-05
socialism : 0.00010179359
bowling : 0.000101502854
moore's : 0.00010254846
inaccurate : 9.826289e-05
genocide : 9.918726e-05
lockheed : 0.000104291285
heston : 0.00010817512
filmmaking : 0.000105953346
mentions : 0.000106469015
kayla : 0.000107454245
rolland : 9.932171e-05
distortion : 0.0001073636
faked : 0.000103699706
heston's : 0.00010261743
assertion : 0.00010422736
revitalization : 0.00010293265
maya : 0.000108254506
guatemala : 0.00010119445
guambiano : 9.0728754e-05
mayans : 0.00010411789
mayan : 9.764657e-05
rhetoric : 9.519073e-05
pan-